<a href="https://colab.research.google.com/github/anudeep22003/nlp-training/blob/main/(chapter_9)_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk import word_tokenize, re
import matplotlib.pyplot as plt
from pprint import pprint
nltk.download('brown')
nltk.download('treebank')
nltk.download('words')


from nltk.corpus import conll2000
nltk.download('conll2000')

import math 
import random

from nltk.corpus import treebank
nltk.download('treebank')

from collections import defaultdict

nltk.download('ppattach')


nltk.download('book_grammars')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package ppattach to /root/nltk_data...
[nltk_data]   Unzipping corpora/ppattach.zip.
[nltk_data] Downloading package book_grammars to /root/nltk_data...
[nltk_data]   Unzipping grammars/book_grammars.zip.


True

# Exercise 1 

☼ What constraints are required to correctly parse word sequences like I am happy and she is happy but not *you is happy or *they am happy? Implement two solutions for the present tense paradigm of the verb be in English, first taking Grammar (6) as your starting point, and then taking Grammar (18) as the starting point.

In [5]:
sentc_1 = "I am happy".split()
sentc_2 = "she is happy".split()

sentw_1 = "you is happy".split()
sentw_2 = "they am happy".split()

grammar = nltk.CFG.fromstring("""

  S -> NP_1_SG VP_1_SG      
  S -> NP_3_SG VP_3_SG      
  S -> NP_3_PL VP_3_PL      
  S -> NP_2_SG VP_2_SG      

  NP_1_SG -> RB_1_SG 
  VP_1_SG -> V_1_SG N                                                                                     

  NP_3_SG -> RB_3_SG
  VP_3_SG -> V_3_SG N

  NP_2_SG -> RB_2_SG 
  VP_2_SG -> V_2_SG N

  NP_3_PL -> RB_3_PL
  VP_3_PL -> V_3_PL N

  RB_1_SG -> "I"
  RB_3_SG -> "she"
  RB_2_SG -> "you"
  RB_3_PL -> "they"
  
  N -> "happy"

  V_1_SG -> "am"
  V_3_SG -> "is"
  V_2_SG -> "are"
  V_3_PL -> "are"

""")

rdp = nltk.RecursiveDescentParser(grammar)

for tree in rdp.parse(sentc_1):
  print(tree)

(S (NP_1_SG (RB_1_SG I)) (VP_1_SG (V_1_SG am) (N happy)))


In [ ]:
grammar2 = nltk.CFG.fromstring("""
  S -> NP[AGR=?n] VP[AGR=?n]
  NP[AGR=?n] = RB[AGR=?n]
  VP[TENSE=?t, AGR=?n] -> V[TENSE=?t, AGR=?n] N

  RB[AGR=[PER=1, NUM=SG]] -> "I"
  RB[AGR=[PER=2, NUM=SG]] -> "you"
  RB[AGR=[PER=3, NUM=SG]] -> "she"
  RB[AGR=[PER=3, NUM=PL]] -> "they"

  V[TENSE = pres, AGR=[PER=1, NUM=SG]] -> "am"
  V[TENSE = pres, AGR=[PER=2, NUM=SG]] -> "are"
  V[TENSE = pres, AGR=[PER=3, NUM=SG]] -> "is"
  V[TENSE = pres, AGR=[PER=3, NUM=PL]] -> "are"

  N -> "happy"

""")


# Exercise 2

☼ Develop a variant of grammar in 1.1 that uses a feature count to make the distinctions shown below:

```
(54)		
a.		The boy sings.
b.		*Boy sings.

(55)		
a.		The boys sing.
b.		Boys sing.

(56)		
a.		The boys sing.
b.		Boys sing.

(57)		
a.		The water is precious.
b.		Water is precious.

```

--- 

Handled all the cases except for the `Boy sings.` being wrong. It is not clear to me if that structure is wrong, and if it is how do I diffferentiate that case from the others. 

In [ ]:
"""
############################
## Grammar Produtions ##
############################

## S Expansion Productions

S -> NP[NUM=?N] VP[NUM=?N]

NP[NUM=?n] -> Det N[NUM=?n]
NP[NUM=pl] -> N[NUM=pl]

VP[NUM=?n] -> IV[NUM=?n]
VP[NUM=?n] -> TV[NUM=?n] N

############################
#### Lexical Produtions ####
############################

Det -> 'the'
N[NUM=sg] -> 'boy' | 'water'
N[NUM=pl] -> 'boys'
N -> 'precious'

IV[NUM=sg] -> 'sings'
IV[NUM=pl] -> 'sing'
TV[NUM=sg] -> 'is'

"""

# Exercise 3

☼ Write a function subsumes() which holds of two feature structures fs1 and fs2 just in case fs1 subsumes fs2.

In [13]:
fs_agr = nltk.FeatStruct(PER=3, NUM = 'pl', GND = 'fem')
fs_tense_pres = nltk.FeatStruct(TENSE = 'pres')
fs_id = nltk.FeatStruct(NAME = 'LEE')
fs_address = nltk.FeatStruct(HOUSE_NUMBER=79, STREET = "5th MAIN", CITY = "BANGALORE")
fs_combo = nltk.FeatStruct(fs_id, ADDRESS = fs_address)

def subsumes(fs1, fs2):
  if fs1.unify(fs2) == fs2:
    return fs1, fs2
    
  elif fs1.unify(fs2) == fs1:
    return fs2, fs1 
  
  else:
    return fs1, fs2, fs1.unify(fs2)
  

print(subsumes(fs_id, fs_address))
print(fs_combo)

([NAME='LEE'], [CITY='BANGALORE', HOUSE_NUMBER=79, STREET='5th MAIN'], [CITY='BANGALORE', HOUSE_NUMBER=79, NAME='LEE', STREET='5th MAIN'])
[           [ CITY         = 'BANGALORE' ] ]
[ ADDRESS = [ HOUSE_NUMBER = 79          ] ]
[           [ STREET       = '5th MAIN'  ] ]
[                                          ]
[ NAME    = 'LEE'                          ]
